In [2]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# features 
features = [
    'temp',
    'precip',
    'rel_humidity',
    'wind_spd',
    'atmos_press',
    'wind_vec', 
    'target'
    ]

for feat in features:
    exec(f'{feat} = pd.read_csv("./data/" + str(feat) + ".csv", index_col=[0])')




In [17]:
for feat in features:
    df = locals()[feat]
    name = df.columns[0][4:-1]
    print(f'DF: {name}:  {df.shape}')

DF: temp: /t/t (14536, 121)
DF: precip: /t/t (14536, 121)
DF: rel_humidity: /t/t (14536, 121)
DF: wind_spd: /t/t (14536, 121)
DF: atmos_press: /t/t (14536, 121)
DF: wind_vec: /t/t (14536, 121)
DF: e: /t/t (14536, 1)


## Model forecast for each feature

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [19]:
def train_predict(model, df, target):
    # data split
    X_train, X_test, y_train, y_test = train_test_split(df, target,random_state=44)
    #model
    
    model.fit(X_train, y_train)
    # predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    rsme_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rsme_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

    test_results[str(model) + "_"+ df.columns[0]] = [rsme_train, rsme_test]

    print(df.columns[0])
    print (f'[{rsme_train:.2f}, {rsme_test:.2f}]')
    return y_pred_train



In [24]:
features.remove('target')

In [25]:
test_results = {}
linear = LinearRegression()
for dframe in features:
    df = locals()[dframe]
    name = df.columns[0]

    train_predict(linear, df, target)


new_temp0
[38.04, 38.50]
new_precip0
[40.44, 41.09]
new_rel_humidity0
[38.57, 39.10]
new_wind_spd0
[38.95, 39.39]
new_atmos_press0
[38.58, 38.92]


ValueError: could not convert string to float: '[0.13, -0.99]'

In [26]:
pd.DataFrame(test_results).T

,0,1
LinearRegression()_new_temp0,38.044771,38.503181
LinearRegression()_new_precip0,40.441800,41.090318
LinearRegression()_new_rel_humidity0,38.566880,39.104745
LinearRegression()_new_wind_spd0,38.951910,39.394749
LinearRegression()_new_atmos_press0,38.578760,38.918730


temperatur has the best fit


# dnn

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
def model_compile_and_fit(df, target, max_epochs=30):
    # Get optimizer
    #optimizer=tf.keras.optimizers.Adam()
    # data split
    X_train, X_test, y_train, y_test = train_test_split(df, target,random_state=44)

    # model 
    with tf.device('/cpu:0'):
      model = tf.keras.Sequential([
            tf.keras.layers.Dense(32,kernel_initializer = 'uniform', activation='relu', input_dim = 121),
            #tf.keras.layers.Dense(121,kernel_initializer = 'uniform', activation='relu', input_dim = 121),
            tf.keras.layers.Dense(1,kernel_initializer = 'uniform')
            ])

    N_VAL =  len(X_test)
    N_TRAIN = len(X_train)
    BATCH_SIZE = 96
    STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
    EPOCHS = 500

    # model.compile
    model.compile(optimizer='Adam',
                metrics='mse', # [tf.keras.metrics.RootMeanSquaredError()]
                loss='mae')
    # model.fit
    with tf.device('/cpu:0'):
      train_history = model.fit(X_train, 
                        y_train,
                        validation_split=0.2,
                        verbose=1,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS, 
                        )
   

    # Predict values for test set
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

    print(df.columns[0])
    print (f'[{rmse_train:.2f}, {rmse_test:.2f}]')

    test_results[str(model) + "_"+ df.columns[0]] = [rmse_train, rmse_test]
    pass

In [ ]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]

    model_compile_and_fit(df, target)

In [ ]:
pd.DataFrame(test_results).T